In [1]:
# a board class, can generator random nr x nc board
import random, copy
class Board():
    '''
    generate a nr x nc board
    '''
    def __init__(self, nr = 2, nc = -1):
        if nc < 0:
            nc = nr
        self.nr = nr
        self.nc = nc
        self.board = [['_' for i in range(nc)] for j in range(nr)]
        
    def get_board(self):
        return copy.deepcopy(self.board)
    
    def get_rlist(self):
        self.rlist = []
        for i in range(self.nr):
            row = self.board[i]
            line = []
            mark = False
            for grid in row:
                if grid == 'X':
                    if mark:
                        line[-1] += 1
                    else:
                        line.append(1)
                        mark = True
                else:
                    if mark:
                        mark = False
            self.rlist.append(line if line else [0])
        return copy.deepcopy(self.rlist)
    
    def get_clist(self):
        self.clist = []
        for i in range(self.nc):
            col = [self.board[j][i] for j in range(self.nr)]
            line = []
            mark = False
            for grid in col:
                if grid == 'X':
                    if mark:
                        line[-1] += 1
                    else:
                        line.append(1)
                        mark = True
                else:
                    if mark:
                        mark = False
            self.clist.append(line if line else [0])
        return copy.deepcopy(self.clist)
    
    def set_row(self, idx, l):
        self.board[idx] = l
        return
    
    def set_col(self, idx, l):
        for i in range(self.nc):
            self.board[i][idx] = l[i]
        return
    
    def set_grid(self, ridx, cidx, val):
        self.board[ridx][cidx] = val
        return
    
    def set_board(self, board):
        for i in range(self.nr):
            self.set_row(i, board[i])
        return
    
    def print_board(self):
        rlist, clist = self.get_rlist(), self.get_clist()
        print("board:")
        head = max(map(len, clist))
        for i in range(head):
            line = []
            for j in range(len(clist)):
                line.append(str(clist[j][i-(head-len(clist[j]))]) if i >= head-len(clist[j]) else ' ')
            print('{0: <10} {1: <10}'.format(' ', '  '.join(line)))
        print("-"*(12 + len(clist) * 3))
        for line in zip(rlist, self.board):
            print('{0: <10} {1: <10}'.format(', '.join(map(str,line[0])), ', '.join(line[1])))
        return
    
    def gen_rand_board(self):
        def gen_rand_line(n):
            res = []
            n += 2
            total = 0
            curMark = bool(random.randint(0,1))
            while total < n:
                cur = random.randint(1, n-total)
                mark = 'X' if curMark else 'O'
                res.extend([mark] * cur)
                curMark = not curMark
                total += cur
            return res[1:-1]
        
        for i in range(self.nr):
            row = gen_rand_line(self.nc)
            self.set_row(i, row)
            
        return self.get_board()

In [2]:
# a function to print the board with line restrictions
def bprint(board, rlist, clist):
    print("board:")
    head = max(map(len, clist))
    for i in range(head):
        line = []
        for j in range(len(clist)):
            line.append(str(clist[j][i-(head-len(clist[j]))]) if i >= head-len(clist[j]) else ' ')
        print('{0: <10} {1: <10}'.format(' ', '  '.join(line)))
    print("-"*(12 + len(clist) * 3))
    for line in zip(rlist, board):
        print('{0: <10} {1: <10}'.format(', '.join(map(str,line[0])), ', '.join(line[1])))
    return

In [3]:
# a function to prefill a board according to line restrictions
def prefill(board, h, v, n):
    def build(l, n):
        if l == [0]:
            return ['O'] * n
        res = ['_' for i in range(n)]
        presum, postsum = [0], [0]
        for _v in l:
            presum.append(presum[-1] + _v + 1)
        for _v in l[::-1]:
            postsum.append(postsum[-1] + _v + 1)
        postsum.pop()
        postsum = postsum[::-1]
        for i,_v in enumerate(l):
            low_e = presum[i] + (_v-1)
            high_s = n-1 - postsum[i] - (_v-1)
            for j in range(max(high_s, 0), min(low_e+1, n)):
                res[j] = 'X'
        return res
    for i in range(n):
        hm = build(h[i], n)
        vm = build(v[i], n)
        for j in range(n):
            board[i][j] = hm[j] if board[i][j] == '_' else board[i][j]
            board[j][i] = vm[j] if board[j][i] == '_' else board[j][i]
    return

In [4]:
# a fucntion pos_lines (l, bl, n) return a list of all possible line outcomes, considering restriction l and board line bl
# test if the possible line conflicts with bl
def valid(l, bl, n):
    for i in range(n):
        if bl[i] == '_':
            continue
        if l[i] != bl[i]:
            return False
    return True

# all possible lines from l in n grids
def perm(l, n):
    def dfs(l, n, idx, ls, dres):
        sls = sum(ls)
        if sls > n:
            return
        if idx == len(l):
            dres.append(ls + [n-sls])
            return
        if idx == 0:
            for g in range(n-l[idx]+1):
                dfs(l, n, idx+1, ls + [g, l[idx]], dres)
        else:
            for g in range(1, n-l[idx]+1-sls):
                dfs(l, n, idx+1, ls + [g, l[idx]], dres)
        return
    def convt(l, n):
        res = ['O'] * n
        i = l[0]
        ln = len(l)
        for li in range(1, ln, 2):
            for j in range(l[li]):
                res[i+j] = "X"
            i += (l[li] + l[li+1])
        return res
    if l == [0]:
        return [["O"] * n]
    elif l == [n]:
        return [["X"] * n]
    dres = []
    dfs(l, n, 0, [], dres)
    res = [convt(_, n) for _ in dres]
    return res

def pos_lines(l, bl, n):
    '''
    return a list of all possible line outcomes, 
    considering restriction l and board line bl
    '''
    
#     bl represents the line on board
#     'O' -> cannot be mark
#     '_' -> undetermined
#     'X' -> marked
    
    return [_ for _ in perm(l, n) if valid(_, bl, n)]

In [5]:
# a function to generate a line that does not conflict any possible lines.
def update(ls):
    n, m = len(ls), len(ls[0])
    count = [0]*m
    for i in range(n):
        for j in range(m):
            count[j] += 1 if ls[i][j] == 'X' else -1
    res = []
    for i in range(m):
        if count[i] == n:
            res.append('X')
        elif count[i] == -n:
            res.append('O')
        else:
            res.append('_')
    return res

In [6]:
# a function to solve a board, if solved, return completed board; if not solved, return the partially finished board
def solve(rlist, clist, verbose = False):
    nr, nc = len(rlist), len(clist)
    if nr != nc:
        print("Line restriction input error. Lengths not equal!")
        return False, None
    if sum([sum(_) for _ in rlist]) != sum([sum(_) for _ in clist]):
        print("Line restriction input error. hsum and v sum not equal!")
        return False, None
    board = [['_' for i in range(nc)] for j in range(nr)]
    def colum(i):
        return [board[j][i] for j in range(nr)]
    
    
    prefill(board, rlist, clist, nr)
    
    rposs, cposs = [], []
    for i in range(nr):
        rposs.append(pos_lines(rlist[i], board[i], nr))
        cposs.append(pos_lines(clist[i], colum(i), nr))
    
    if verbose:
        print([len(l) for l in rposs])
        print([len(l) for l in cposs])
    
    finished = 0
    changelessloop = 0
    loop = 1
    while changelessloop < 2:
        add = 0
        for i in range(nr):
            if not rposs[i]:
                continue
            rpos = pos_lines(rlist[i], board[i], nr)
            board[i] = update(rpos)
            if len(rpos) == 1 or len(rpos) < len(rposs[i]):
                rposs[i] = rpos if len(rpos) > 1 else None
            add += 1 if not rposs[i] else 0
            
        for i in range(nc):
            if not cposs[i]:
                continue
            cpos = pos_lines(clist[i], colum(i), nc)
            updated = update(cpos)
            for j in range(nr):
                board[j][i] = updated[j]
            if len(cpos) == 1 or len(cpos) < len(cposs[i]):
                cposs[i] = cpos if len(cpos) > 1 else None
            add += 1 if not cposs[i] else 0
        
        changelessloop = changelessloop + 1 if add == 0 else 0
        finished += add
        
        if verbose:
            print("this is loop {}, we finished {} lines. The possibilities are:".format(loop, finished))
            print([len(l) if l else 0 for l in rposs])
            print([len(l) if l else 0 for l in cposs])
            bprint(board, rlist, clist)
        
        loop += 1
        if finished == 2 * nr:
            print("BOARD COMPLETE   " * 3)
            return True, board
            
    return False, board

In [7]:
# a function to generate no more than N possible finished boards
# this can be an expensive search as each line can have 100 possibilities to be tested
import copy
def complete_board(board, rlist, clist, N=1):
    def valid(board, cposs):
        for i, cpos in enumerate(cposs):
            bl = ''.join([board[j][i] for j in range(len(board))])
            match = False
            for pl in cpos:
                if ''.join(pl) == bl:
                    match = True
                    break
            if not match:
                return False
        return True
    def dfs(board, rposs, cposs, idx, res, nr, N):
        if idx >= nr:
            if valid(board, cposs):
                res.append(copy.deepcopy(board))
                return True
            else:
                return False
        if len(res) >= N:
            return True
        if len(rposs[idx]) == 1:
            return dfs(board, rposs, cposs, idx+1, res, nr, N)
#         line_snap = copy.deepcopy(board[idx])
        flag = False
        for pl in rposs[idx]:
            board[idx] = pl
            if dfs(board, rposs, cposs, idx+1, res, nr, N):
                flag = True
        return flag
            
    rposs, cposs = [], []
    nr = len(rlist)
    def colum(i):
        return [board[j][i] for j in range(nr)]
    for i in range(nr):
        rposs.append(pos_lines(rlist[i], board[i], nr))
        cposs.append(pos_lines(clist[i], colum(i), nr))
    res = []
    dfs(board, rposs, cposs, 0, res, nr, N)
    return res

In [8]:
# a function to generate a list of n sizexsize boards
def gen_boards(size, n):
    army_of_boards = []
    for i in range(n):
        b = Board(size)
        b.gen_rand_board()
        army_of_boards.append(b)
    return army_of_boards

# a test function
import time
def test(size, n):
    army_of_boards = gen_boards(size, n)
    count = 0
    start = time.time()
    for b in army_of_boards:
        print('Starting New Board')
        rlist, clist = b.get_rlist(), b.get_clist()
        solved, board = solve(rlist, clist)
        
        bprint(board, rlist, clist)
        print()
        count += 1 if solved else 0
        
#         may not want to deal with the dfs as it can take long
#         if not solved:
#             boards = complete_board(board, rlist, clist)
#             for i, bd in enumerate(boards):
#                 print("*"*10 ,"  Possible Solution {}  ".format(i+1), "*"*10)
#                 bprint(boards[0], rlist, clist)
#                 print()
    end = time.time()
    print("{} out of {} boards has one and only solution".format(count, n))
    print("Solved {} {}X{} boards in {} seconds, average {:.2f} seconds/board".format(n, size, size, end-start, (end-start)/n))

In [9]:
test(3, 30)

Starting New Board
board:
           1  0  1   
---------------------
1          _, O, _   
1          _, O, _   
0          O, O, O   

Starting New Board
BOARD COMPLETE   BOARD COMPLETE   BOARD COMPLETE   
board:
           3  2  2   
---------------------
1          X, O, O   
3          X, X, X   
3          X, X, X   

Starting New Board
board:
           1  1  1   
---------------------
0          O, O, O   
2          _, X, _   
1          _, O, _   

Starting New Board
BOARD COMPLETE   BOARD COMPLETE   BOARD COMPLETE   
board:
              1  1   
           1  1  1   
---------------------
3          X, X, X   
0          O, O, O   
2          O, X, X   

Starting New Board
BOARD COMPLETE   BOARD COMPLETE   BOARD COMPLETE   
board:
           2  1  3   
---------------------
2          O, X, X   
1, 1       X, O, X   
1, 1       X, O, X   

Starting New Board
BOARD COMPLETE   BOARD COMPLETE   BOARD COMPLETE   
board:
           2  1  3   
---------------------
3          X, X

In [10]:
test(10, 30)

Starting New Board
BOARD COMPLETE   BOARD COMPLETE   BOARD COMPLETE   
board:
                             2         
           2        2     2  1  5  5  1
           2  2  2  1  2  2  1  1  1  1
           1  2  2  2  4  1  2  2  1  3
------------------------------------------
9          X, X, X, X, X, X, X, X, X, O
10         X, X, X, X, X, X, X, X, X, X
2          O, O, O, O, O, O, O, X, X, O
3, 1, 3    X, X, X, O, O, X, O, X, X, X
9          X, X, X, X, X, X, X, X, X, O
1          O, O, O, O, X, O, O, O, O, O
1, 6       X, O, O, X, X, X, X, X, X, O
2, 1       O, O, O, X, X, O, O, O, O, X
4          O, O, O, O, O, O, X, X, X, X
2, 1       O, O, O, O, O, O, X, X, O, X

Starting New Board
BOARD COMPLETE   BOARD COMPLETE   BOARD COMPLETE   
board:
                          2            
           2  4  5  4  5  1            
           1  2  2  2  2  2  2  1  1  1
           2  1  1  1  1  1  4  4  3  3
------------------------------------------
5          X, X, X, X, X, O, O, O, O,

In [11]:
test(15, 30)

Starting New Board
BOARD COMPLETE   BOARD COMPLETE   BOARD COMPLETE   
board:
              1  1  1  1  1  1  1  1  1  1           2
           3  1  1  1  1  1  1  1  1  1  1     2     1
           1  1  1  1  5  5  5  3  3  3  3  3  2  2  1
           2  2  1  1  1  1  1  1  1  1  1  3  3  3  1
           1  2  2  2  2  2  2  2  1  1  1  3  3  4  4
---------------------------------------------------------
3, 3       O, O, O, O, X, X, X, O, O, O, O, O, X, X, X
3          O, O, O, O, O, O, O, O, O, O, O, O, X, X, X
12         X, X, X, X, X, X, X, X, X, X, X, X, O, O, O
1, 2       X, O, O, O, O, O, O, O, O, O, O, X, X, O, O
13         X, X, X, X, X, X, X, X, X, X, X, X, X, O, O
3, 1       O, O, O, O, X, X, X, O, O, O, O, O, O, O, X
12         X, X, X, X, X, X, X, X, X, X, X, X, O, O, O
11         O, O, O, O, X, X, X, X, X, X, X, X, X, X, X
10         O, O, O, O, X, X, X, X, X, X, X, X, X, X, O
2, 3       X, X, O, O, O, O, O, O, O, O, O, O, X, X, X
11         X, X, X, X, X, X, X, X, X, X

BOARD COMPLETE   BOARD COMPLETE   BOARD COMPLETE   
board:
                                            2     1   
           1  1                    1  1  1  2  2  2  2
           1  1              2  2  2  1  2  1  2  1  2
           3  2  1  1  1  1  1  1  1  1  1  2  4  2  1
           1  1  2  1  1  2  2  2  2  5  5  3  2  1  3
---------------------------------------------------------
6          O, O, O, O, O, O, O, O, X, X, X, X, X, X, O
2, 1       O, O, O, O, O, O, O, O, O, O, O, X, X, O, X
2, 2       X, X, O, O, O, O, O, O, O, O, O, O, O, X, X
8          O, O, O, O, O, O, X, X, X, X, X, X, X, X, O
3, 3       O, O, O, O, O, O, X, X, X, O, X, X, X, O, O
0          O, O, O, O, O, O, O, O, O, O, O, O, O, O, O
15         X, X, X, X, X, X, X, X, X, X, X, X, X, X, X
1, 1       O, O, O, O, O, O, O, O, O, O, O, O, X, O, X
1, 3       X, O, O, O, O, O, O, O, O, O, O, X, X, X, O
3, 6       X, X, X, O, O, O, O, O, O, X, X, X, X, X, X
11         X, X, X, X, X, X, X, X, X, X, X, O, O, O, O
10 

BOARD COMPLETE   BOARD COMPLETE   BOARD COMPLETE   
board:
           1  1     1                                 
           3  3  1  1                    1  1     1   
           1  1  3  1  1  1  1  1  1  1  3  1  1  2   
           2  1  3  3  1  1  1  1  1  3  1  2  1  1  7
           1  1  1  1  3  2  2  1  1  1  2  2  1  3  5
           1  1  1  1  1  2  2  2  3  3  1  1  3  1  1
---------------------------------------------------------
1          O, O, O, O, O, O, O, O, O, O, O, O, O, O, X
15         X, X, X, X, X, X, X, X, X, X, X, X, X, X, X
1          O, O, O, O, O, O, O, O, O, O, O, O, O, O, X
13, 1      X, X, X, X, X, X, X, X, X, X, X, X, X, O, X
3, 2, 1    X, X, X, O, O, O, O, O, O, X, X, O, O, O, X
4, 3, 2    X, X, X, X, O, O, O, O, O, X, X, X, O, X, X
4          O, O, O, O, O, O, O, O, O, O, O, X, X, X, X
11         X, X, X, X, X, X, X, X, X, X, X, O, O, O, O
5, 2       O, O, X, X, X, X, X, O, O, O, O, O, O, X, X
5, 1       X, X, X, X, X, O, O, O, O, O, O, O, O, O, X
1, 

In [12]:
test(15, 30)

Starting New Board
BOARD COMPLETE   BOARD COMPLETE   BOARD COMPLETE   
board:
           2                                          
           2  5  5                                    
           1  1  1  5                 6  7  4     1  1
           2  2  1  1  6  6        8  1  1  1  4  2  2
           1  1  1  2  1  1  6  6  2  2  1  1  3  1  2
           1  1  1  1  4  4  5  5  1  1  1  4  2  3  1
---------------------------------------------------------
4, 6       O, O, O, O, X, X, X, X, O, X, X, X, X, X, X
13         X, X, X, X, X, X, X, X, X, X, X, X, X, O, O
15         X, X, X, X, X, X, X, X, X, X, X, X, X, X, X
14         O, X, X, X, X, X, X, X, X, X, X, X, X, X, X
11         X, X, X, X, X, X, X, X, X, X, X, O, O, O, O
11         X, X, X, X, X, X, X, X, X, X, X, O, O, O, O
1, 2       O, O, O, O, O, O, O, O, X, O, X, X, O, O, O
9          X, X, X, X, X, X, X, X, X, O, O, O, O, O, O
9          O, O, O, O, O, O, X, X, X, X, X, X, X, X, X
2, 5, 1, 1 X, X, O, X, X, X, X, X, O, O

BOARD COMPLETE   BOARD COMPLETE   BOARD COMPLETE   
board:
                                            1  1  1   
                                            1  1  2   
           3  3  3  3  3  3  3  3  3  3  3  1  1  1  2
           3  3  3  3  3  3  3  3  3  2  2  1  2  1  1
           2  2  2  2  3  3  3  3  3  3  3  1  1  1  2
---------------------------------------------------------
1          O, O, O, O, O, O, O, O, O, O, O, O, O, O, X
15         X, X, X, X, X, X, X, X, X, X, X, X, X, X, X
11         X, X, X, X, X, X, X, X, X, X, X, O, O, O, O
14         X, X, X, X, X, X, X, X, X, X, X, X, X, X, O
1          O, O, O, O, O, O, O, O, O, O, O, O, O, X, O
0          O, O, O, O, O, O, O, O, O, O, O, O, O, O, O
0          O, O, O, O, O, O, O, O, O, O, O, O, O, O, O
5          O, O, O, O, O, O, X, X, X, X, X, O, O, O, O
11         X, X, X, X, X, X, X, X, X, X, X, O, O, O, O
9, 4       X, X, X, X, X, X, X, X, X, O, O, X, X, X, X
6          X, X, X, X, X, X, O, O, O, O, O, O, O, O, O
3  

BOARD COMPLETE   BOARD COMPLETE   BOARD COMPLETE   
board:
           2        1  1  1                           
           1  2  2  1  1  1     1                     
           1  1  1  1  1  1  1  1  1  1  1  1         
           1  1  1  2  1  1  1  1  1  2  2  2  4  5  3
           3  3  3  3  3  3  3  2  2  2  3  3  4  2  3
           1  1  1  1  1  1  1  1  1  2  2  1  1  1  1
---------------------------------------------------------
3          X, X, X, O, O, O, O, O, O, O, O, O, O, O, O
6, 1       X, X, X, X, X, X, O, O, O, O, O, O, O, X, O
6          O, O, O, O, O, O, O, O, X, X, X, X, X, X, O
6, 1, 3    X, X, X, X, X, X, O, X, O, O, O, O, X, X, X
6          O, O, O, O, O, O, O, O, O, X, X, X, X, X, X
15         X, X, X, X, X, X, X, X, X, X, X, X, X, X, X
0          O, O, O, O, O, O, O, O, O, O, O, O, O, O, O
5          O, O, O, X, X, X, X, X, O, O, O, O, O, O, O
1, 1, 1    X, O, O, X, O, O, O, O, O, O, O, O, X, O, O
5          O, O, O, O, O, O, O, O, O, O, X, X, X, X, X
15 

BOARD COMPLETE   BOARD COMPLETE   BOARD COMPLETE   
board:
                             1        1               
                          1  1  1  1  1               
           1     1  1  1  1  2  1  1  1              2
           1  1  1  1  1  2  1  2  2  1        1  1  4
           3  3  1  1  1  1  1  1  1  2  3  3  3  2  1
           5  5  5  5  4  3  1  1  2  1  7  8  3  2  2
---------------------------------------------------------
12         X, X, X, X, X, X, X, X, X, X, X, X, O, O, O
2          O, O, O, O, O, O, O, O, O, O, X, X, O, O, O
8, 1       O, O, O, O, O, X, X, X, X, X, X, X, X, O, X
1, 1       X, O, O, O, O, O, O, O, O, O, O, O, O, O, X
2          O, O, O, O, O, O, O, O, O, O, O, X, X, O, O
8, 1       O, O, O, O, O, X, X, X, X, X, X, X, X, O, X
9, 3, 1    X, X, X, X, X, X, X, X, X, O, X, X, X, O, X
2, 2, 2    X, X, O, O, O, O, O, O, O, O, X, X, O, X, X
13, 1      X, X, X, X, X, X, X, X, X, X, X, X, X, O, X
4          O, O, O, O, O, O, O, O, O, O, X, X, X, X, O
15 

In [13]:
test(15, 40)

Starting New Board
BOARD COMPLETE   BOARD COMPLETE   BOARD COMPLETE   
board:
                                               1     1
                                      3        1     1
                 4  4              2  2     6  1  2  2
           2  3  1  1  4  4  4  3  2  1  6  1  1  2  1
           1  1  1  1  1  1  1  1  1  1  1  1  2  1  1
           1  1  1  1  3  3  3  1  1  1  1  1  2  4  1
---------------------------------------------------------
7, 2, 1    X, X, X, X, X, X, X, O, O, X, X, O, O, O, X
14         X, X, X, X, X, X, X, X, X, X, X, X, X, X, O
11, 2      O, X, X, X, X, X, X, X, X, X, X, X, O, X, X
6, 3       O, O, X, X, X, X, X, X, O, O, X, X, X, O, O
3, 2       O, O, O, O, O, O, O, O, O, X, X, X, O, X, X
7          O, O, O, O, O, O, O, O, X, X, X, X, X, X, X
1, 1       O, O, O, O, O, O, O, O, X, O, O, X, O, O, O
7, 2       X, X, X, X, X, X, X, O, O, O, O, O, X, X, O
3          O, O, O, O, O, O, O, O, O, X, X, X, O, O, O
1          O, O, O, O, O, O, O, O, O, O

BOARD COMPLETE   BOARD COMPLETE   BOARD COMPLETE   
board:
                                                  1   
           1  1        2           2  2           1   
           1  1  2  2  1  2  1  1  1  1           1  1
           1  1  1  1  1  3  5  1  3  3  2  1  3  2  3
           1  1  1  1  1  1  1  5  1  1  5  6  4  1  2
           1  1  1  1  1  1  1  1  1  1  3  1  1  1  2
---------------------------------------------------------
6, 4       X, X, X, X, X, X, O, O, X, X, X, X, O, O, O
9, 3       O, O, X, X, X, X, X, X, X, X, X, O, X, X, X
2, 1       X, X, O, O, O, O, O, O, O, O, O, O, X, O, O
8          O, O, O, O, O, O, O, X, X, X, X, X, X, X, X
2, 1       O, O, O, O, O, O, O, O, O, O, X, X, O, O, X
15         X, X, X, X, X, X, X, X, X, X, X, X, X, X, X
8          O, O, O, O, O, X, X, X, X, X, X, X, X, O, O
14         X, X, X, X, X, X, X, X, X, X, X, X, X, X, O
2, 4       O, O, O, O, O, O, X, X, O, O, O, X, X, X, X
10, 1      X, X, X, X, X, X, X, X, X, X, O, O, O, O, X
1  

board:
           1  1  2  2  2  1  2                        
           1  1  1  1  1  1  1  2  1  1  2            
           1  1  1  1  1  1  1  1  1  1  2  1  1      
           1  1  1  1  1  1  1  1  1  1  1  2  2     3
           2  2  2  2  2  1  1  1  1  2  1  1  1  4  1
           1  1  1  1  1  1  1  3  3  2  3  4  1  1  2
---------------------------------------------------------
1          O, O, O, O, O, O, O, O, O, O, X, O, O, O, O
3, 8       O, O, X, X, X, O, X, X, X, X, X, X, X, X, O
8, 2       X, X, X, X, X, X, X, X, O, O, O, O, O, X, X
5          O, O, O, O, O, O, O, O, O, O, X, X, X, X, X
15         X, X, X, X, X, X, X, X, X, X, X, X, X, X, X
0          O, O, O, O, O, O, O, O, O, O, O, O, O, O, O
11         X, X, X, X, X, X, X, X, X, X, X, O, O, O, O
1          O, O, O, O, O, O, O, O, O, O, O, O, _, _, _
10, 1      X, X, X, X, X, X, X, X, X, X, O, O, _, _, _
3, 1       O, O, O, O, O, O, O, O, O, X, X, X, O, _, _
5          X, X, X, X, X, O, O, O, O, O, O, O, O, O, O


7          O, O, O, O, O, O, O, X, X, X, X, X, X, X, O
10, 1      X, X, X, X, X, X, X, X, X, X, O, O, O, O, X

Starting New Board
BOARD COMPLETE   BOARD COMPLETE   BOARD COMPLETE   
board:
           2        2  2  1  1                        
           1  2  2  2  2  2  2  1  1  1  1  1  2  1   
           1  2  2  2  2  2  2  2  2  2  2  1  2  1  1
           2  4  4  1  1  1  1  1  1  1  2  3  1  1  3
           2  2  2  2  2  3  3  3  3  3  3  3  2  5  3
---------------------------------------------------------
5          X, X, X, X, X, O, O, O, O, O, O, O, O, O, O
12         X, X, X, X, X, X, X, X, X, X, X, X, O, O, O
1          O, O, O, O, O, O, O, O, O, O, O, O, X, O, O
14         O, X, X, X, X, X, X, X, X, X, X, X, X, X, X
11         X, X, X, X, X, X, X, X, X, X, X, O, O, O, O
2          O, O, O, O, O, O, O, O, O, O, O, O, O, X, X
13, 1      X, X, X, X, X, X, X, X, X, X, X, X, X, O, X
6, 5       O, X, X, X, X, X, X, O, O, O, X, X, X, X, X
3, 1       X, X, X, O, O, O, O, O, O, 

BOARD COMPLETE   BOARD COMPLETE   BOARD COMPLETE   
board:
              2  2                                    
           2  1  1  3  3  2                           
           1  1  1  1  1  1  2  1  1  1  1  2  3     2
           2  1  1  1  1  1  1  1  2  2  1  1  1  4  1
           1  1  1  1  1  1  1  1  1  1  1  3  1  3  2
           2  2  2  2  2  2  3  3  2  2  4  2  2  2  1
---------------------------------------------------------
7, 1       X, X, X, X, X, X, X, O, O, O, O, O, O, X, O
14         X, X, X, X, X, X, X, X, X, X, X, X, X, X, O
2, 4       O, O, O, X, X, O, O, O, O, O, O, X, X, X, X
2, 3       O, X, X, O, O, O, O, O, O, O, O, O, X, X, X
1          O, O, O, O, O, O, O, O, O, O, O, X, O, O, O
1          X, O, O, O, O, O, O, O, O, O, O, O, O, O, O
5, 5       O, X, X, X, X, X, O, O, O, O, X, X, X, X, X
1, 2, 1, 1 X, O, O, O, O, O, O, O, X, X, O, X, O, X, O
15         X, X, X, X, X, X, X, X, X, X, X, X, X, X, X
1          O, O, O, O, O, O, O, O, O, O, O, O, O, O, X
11 

In [ ]:
aob = gen_boards(15, 300)
unsolved = []
for b in aob:
    rlist, clist = b.get_rlist(), b.get_clist()
    solved, board = solve(rlist, clist)
    if not solved:
        unsolved.append(b)
print(len(unsolved))

In [ ]:
import time
for b in unsolved:
    rlist, clist = b.get_rlist(), b.get_clist()
    solved, board = solve(rlist, clist)
    print('try to solve board:')
    print(rlist)
    print(clist)
    bprint(board, rlist, clist)
    start = time.time()
    boards = complete_board(board, rlist, clist)
    end = time.time()
    print("took {} seconds".format(end-start))
    for i, bd in enumerate(boards):
        print("*"*10, "  Possible Solution {}  ".format(i+1), "*"*10)
        bprint(bd, rlist, clist)
    

In [ ]:
rlist, clist = [[2, 1], [5], [1], [6, 1], [5, 1], [13, 1], [3, 1, 1], [6], [6, 4], [6], [1, 3], [7], [0], [15], [15]], [[1, 2, 1, 1, 2], [1, 2, 1, 1, 2], [2, 2, 2, 2], [2, 2, 1, 2], [2, 2, 1, 2], [1, 1, 2, 1, 2], [1, 1, 1, 1, 1, 2], [1, 1, 1, 1, 2], [1, 1, 2, 1, 2], [1, 1, 2, 2, 2], [2, 2, 2, 2], [1, 1, 2, 2], [2, 3, 2], [1, 2, 2], [1, 1, 2, 1, 2]]
solved, board = solve(rlist, clist)
bprint(board, rlist, clist)
rposs, cposs = [], []
nr = len(rlist)
def colum(i):
    return [board[j][i] for j in range(nr)]
for i in range(nr):
    rposs.append(pos_lines(rlist[i], board[i], nr))
    cposs.append(pos_lines(clist[i], colum(i), nr))
    print(i, len(rposs[-1]), len(cposs[-1]))